In [44]:
import sys, os

# DWave Credentials
dwave_solver = "DW_2000Q_2_1"
api_key = 'DEV-3142f58be13a0aacd44657b15b8fb9832ba3bdac'

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

import dwave_networkx as dnx

### Use NetworkX graphic package to create a signed network
import networkx as nx

# Select a D-Wave system and handle mapping from problem graph to sampler graph
sampler = EmbeddingComposite(DWaveSampler(solver=dwave_solver, token=api_key))

In [36]:
# Import graph node labels
node_labels = []
with open('CryptoSymbols.csv') as f:
    for line in f.readlines():
        line = line.rstrip() 
        node_labels.append(line)

In [37]:
# Import correlation data
import scipy.io as sio

corr_matrices = sio.loadmat('S_mat.mat').get('S_mat')

In [45]:
# Plot Frustration score over time
frustration_array = []

for i in range(1,36):
    G = nx.Graph()
    G.add_nodes_from(node_labels)
    corr_matrix = corr_matrices[i*20,:,:]
    
    # Add weighted edges to graph
    for i in range(27):
        for j in range(i+1, 27):
            if corr_matrix[i,j] < 0:
                G.add_edge(node_labels[i], node_labels[j], sign=-1)
            elif corr_matrix[i,j] > 0:
                G.add_edge(node_labels[i], node_labels[j], sign=1)
    
    try:
        imbalance, bicoloring = dnx.structural_imbalance(G, sampler)
        frustration_array.append(len(list(imbalance.keys()))/G.number_of_edges())
    except:
        print(sys.exc_info()[0])
        print(i)

<class 'RuntimeError'>
